In [64]:
import duckdb

conn = duckdb.connect("../rewards_dbt/rewards_db.duckdb")


# 
- Which brand has the most spend among users who were created within the past 6 months?
- Which brand has the most transactions among users who were created within the past 6 months?

In [ ]:
core_receipts_df = conn.sql("""
    select * exclude(rewards_receipt_item_list)
    from
        core.receipts
    order by
        1
""").df()

display(core_receipts_df)


In [ ]:
core_receipts_item_list_df = conn.sql("""
    select
        *                     
    from
        core.receipts_item_list
""").df()
display(core_receipts_item_list_df)


To get the brand, we need to join with the brands table using either:
- barcode
- brand_code

Assumptions:
- Since we're looking for total spend and not rewards receipt status, all statuses will be considered.
- The "past 6 months" will be defined as (timestamp of newest receipt - 6 months).

Column changes
- Use `nullif()` for barcode values of 4011 since this value means "ITEM NOT FOUND"
- `final_price` is chosen over `item_price` due to discounts

Filters
- Remove rows where we don't know the `barcode` or the `brand_code` since one of these has to link us back to a brand.
- Remove rows where we don't know `item_spend`.
- Remove users where their created date is older than newest receipt date minus 6 months.

In [ ]:
brand_spend_df = conn.sql("""
    with item_spend as (
        select
            ril.receipt_id,
            nullif(coalesce(ril.user_flagged_barcode, ril.barcode), 4011) as item_barcode,
            ril.brand_code,
            coalesce(ril.user_flagged_price, ril.final_price) as item_price,
            coalesce(ril.user_flagged_quantity, ril.quantity_purchased) as item_quantity,
            (item_price * item_quantity) as item_spend
        from
            core.receipts_item_list as ril
        where
            (item_barcode is not null
            or ril.brand_code is not null)
            and item_spend is not null
            -- and coalesce(ril.is_deleted, false) != True                   
    ),
    user_item_spend as (
        select
            spend.receipt_id,
            receipts.user_id,
            users.created_date,
            spend.item_spend,
            brands.id as brand_id,
            brands.brand_code
        from
            item_spend as spend
            inner join core.receipts
            on spend.receipt_id = receipts.id
            inner join core.users
            on receipts.user_id = users.id
            left join core.brands
            on spend.item_barcode = brands.barcode
            or spend.brand_code = brands.brand_code
        where
            brands.brand_code is not null
            or brand_id is not null
    ),
    ranks as (
        select distinct
            brand_code,
            sum(item_spend) as total_spend,
            rank() over(order by total_spend desc) as spend_rank,
            count(0) as total_transactions,
            rank() over(order by total_transactions desc) as transaction_rank
        from
            user_item_spend
        where
            -- Filter out users older than newest receipt date minus 6 months.
            created_date >= (select max(create_date) - interval 6 month from core.receipts)
        group by
            brand_code          
    )
    select *
    from
        ranks
    where
        spend_rank = 1
        or transaction_rank = 1


""").df()
display(brand_spend_df)


- `PEPSI` is the brand with the most transactions among users who were created within the past 6 months.
- `CRACKER BARREL` has the most spend among users who were created within the past 6 months.